In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.stats import norm, t
from arelle import Cntlr

import matplotlib.ticker as ticker
import matplotlib.font_manager as font_manager
from matplotlib.ticker import FormatStrFormatter

In [70]:
#Importar base de datos con precios de acciones
df_acciones = pd.read_csv('C:/Users/andrgome/ACCIONES/ACCIONES_COL.csv',encoding='latin-1', parse_dates=['Fecha'],dayfirst=True)
df_mkt_cap = pd.read_csv('C:/Users/andrgome/ACCIONES/CapitalizacionBVC.csv',encoding='latin-1')

                              
#Diccionario con el nombre de las empresa y sus correspondientes datos como por ejemplo: 
#a) el nombre del archivo xbrl con los estados financieros, b)Calificación del emisor, c) Nombre del emisor en la base de datos de acciones
CORPORATIVO = {"GRUPO ARGOS-20200930":["0052482095_0066_000058_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AA+",'GrupoArgosOrd','GRUPO ARGOS S.A.'],
                "ISA-20200930":["0052482070_0260_000034_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Isa','INTERCONEXION ELECTRICA S.A. E.S.P.'],
               "EL CONDOR-20200930":["0052481694_0056_000008_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'ElCondor','CONSTRUCCIONES EL CONDOR S.A'],
               "GEB-20200930":["0052484062_0260_000043_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Geb','GRUPO ENERGIA BOGOTA S.A. E.S.P.'],
               "ECOPETROL-20200930":["0052479219_0260_000036_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Ecopetrol','ECOPETROL S.A.'],
               "PROMIGAS-20200930":["0052483328_0261_000004_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Promigas','PROMIGAS S.A. E.S.P.'],
               "CEMARGOS-20200930":["0052479806_0043_000005_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'CemargosOrd','CEMENTOS ARGOS S.A.'],
               "CELSIA-20200930":["0052481093_0066_000061_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Celsia','CELSIA S.A'],
               "TERPEL-20200930":["0052484776_0053_000022_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Terpel','ORGANIZACION TERPEL S.A'],
               "ENKA-20200930":["0052482051_0039_000021_0000_000000_000000_I-I_2020-09-30.xbrl","2020-10-01","2020-07-01","AAA",'Enka','ENKA DE COLOMBIA S.A.'],
              }



#Crear dataframe para almacenar datos de las empresas
datos_empresas = pd.DataFrame().T

#Iteramos para todos los elementos en el diccionario
for key in CORPORATIVO:
    
    EEFF_EMPRESA = CORPORATIVO[key][0]
    
    #Importamos archivo xbrl
    xbrl = Cntlr.Cntlr().modelManager.load(r'C:/Users/andrgome/DEUDA PRIVADA/KVM Model/EEFF/'+EEFF_EMPRESA)

        
    #Lo convertimos a un dataframe
    factData=pd.DataFrame(data=[(fact.concept.qname,
                               fact.value,
                               fact.isNumeric,
                               fact.contextID,
                               fact.context.isStartEndPeriod,
                               fact.context.isInstantPeriod,
                               fact.context.isForeverPeriod,
                               fact.context.startDatetime,
                               fact.context.endDatetime,
                               fact.unitID) for fact in xbrl.facts])

    #Volvemos texto la columna cero
    factData['Nombre'] = factData.apply(lambda row: str(row[0]),axis=1)
    #Volvemos numérica la columna 1
    factData[1] = pd.to_numeric(factData[1],errors='coerce')
    #Extraemos datos financieros
    #Pasivo corriente 
    Pasivo_corriente = factData[(factData['Nombre']=="ifrs:CurrentLiabilities")&(factData[3]=="_ctx1")|
                        (factData['Nombre']=="ifrs:CurrentLiabilities")&(factData[3]=="CierreTrimestreActual")|
                        (factData['Nombre']=="ifrs:CurrentLiabilities")&(factData[8]==pd.to_datetime(CORPORATIVO[key][1]))][1].values[0]/1000
    #Pasivo no corriente 
    Pasivo_no_corriente = factData[(factData['Nombre']=="ifrs:NoncurrentLiabilities")&(factData[3]=="_ctx1")|
                        (factData['Nombre']=="ifrs:NoncurrentLiabilities")&(factData[3]=="CierreTrimestreActual")|
                        (factData['Nombre']=="ifrs:NoncurrentLiabilities")&(factData[8]==pd.to_datetime(CORPORATIVO[key][1]))][1].values[0]/1000
    
    
    emisor = CORPORATIVO[key][4]

    #Cálculo volatilidad realizada
    df_vol = pd.DataFrame()
    df_vol['Retorno_diario'] = df_acciones[emisor][1:].values/df_acciones[emisor][:-1]-1
    df_vol['Retorno_diario'].dropna(inplace=True)
    df_vol['volatilidad']=df_vol['Retorno_diario'].rolling(window=252*3).std()*252**0.5
    Vol_equity = df_vol['volatilidad'].iloc[-1]
    Market_cap = df_mkt_cap.loc[df_mkt_cap['Emisor']==CORPORATIVO[key][5]]['Capitalizacion Bursatil'].iloc[0]
    
    Activo_inicial = Market_cap + Pasivo_corriente + Pasivo_no_corriente
    Activo = Activo_inicial
    Pasivo = Pasivo_corriente + Pasivo_no_corriente
    Vol_activo_inicial = Vol_equity
    Vol_activo = Vol_activo_inicial
    Equity = Market_cap
    r = 0.0175
    T = 1
    epsilon = 1

    d1 = (np.log(Activo/Pasivo)+(r+0.5*Vol_activo**2)*T)/(Vol_activo*T**0.5)
    d2 = (np.log(Activo/Pasivo)+(r-0.5*Vol_activo**2)*T)/(Vol_activo*T**0.5)

    def function1(x):
        return x - Activo*norm.cdf(d1) + np.exp(-r*T)*norm.cdf(d2)

    def function2(x):
        return Vol_equity - norm.cdf(d1)*(Activo/Equity)*x


    while epsilon > 0.00001:
        sol2 = root(function2,Vol_activo)
        sol1 = root(function1,Activo)

        Equity = sol1.x[0]
        Vol_activo_nueva = sol2.x[0]
        epsilon = Vol_activo_nueva - Vol_activo
        Vol_activo = Vol_activo_nueva
   
    #Crear una lista y agregar características de las empresas
    lista_caracteristicas = []
    lista_caracteristicas.append(key)
    lista_caracteristicas.append(Activo)
    lista_caracteristicas.append(Pasivo)
    lista_caracteristicas.append(Pasivo_corriente)
    lista_caracteristicas.append(Pasivo_no_corriente)    
    lista_caracteristicas.append(Vol_equity)
    lista_caracteristicas.append(Vol_activo)
    lista_caracteristicas.append(Market_cap)
    lista_caracteristicas.append(d1)
    lista_caracteristicas.append(d2)  
    lista_caracteristicas.append(emisor)  
    
    datos_empresas = datos_empresas.append(pd.DataFrame(lista_caracteristicas).T)


In [71]:
datos_empresas
# df_mkt_cap['Emisor'].unique()
# df_acciones

,0,1,2,3,4,5,6,7,8,9
0,GRUPO ARGOS-20200930,12798429.742,2236703.592,269391.652,1967311.94,0.459941,0.379569,10561726.15,4.060507,3.600566
0,ISA-20200930,32068148.459,6591556.889,82.914,6591473.975,0.342101,0.271783,25476591.57,4.846788,4.504687
0,EL CONDOR-20200930,1757712.876,1131654.136,552248.36,579405.776,0.28611,0.106224,626058.74,1.743254,1.457144
0,GEB-20200930,32828903.177,8563052.237,861932.596,7701119.641,0.239799,0.17725,24265850.94,5.796956,5.557157
0,ECOPETROL-20200930,171974545.758,82011217.778,17079390.268,64931827.51,0.396076,0.210854,89963327.98,2.111787,1.71571
0,PROMIGAS-20200930,12839579.495,4047743.115,813967.291,3233775.824,0.312227,0.213806,8791836.38,3.909382,3.597155
0,CEMARGOS-20200930,13708734.584,6150865.974,2088543.706,4062322.268,0.405841,0.226736,7557868.61,2.220805,1.814964
0,CELSIA-20200930,5994038.942,927718.902,231056.718,696662.184,0.241267,0.203925,5066320.04,7.926485,7.685218
0,TERPEL-20200930,5125784.799,3683459.989,919790.037,2763669.952,0.257061,0.077708,1442324.81,1.482027,1.224966
0,ENKA-20200930,339871.331,195054.521,90000.3,105054.221,0.428283,0.194216,144816.81,1.551547,1.123264
